In [3]:
import os
import sys
from pathlib import Path

import pandas as pd
import matplotlib as mp
import sys
import os

# Add the project root to Python path to import from src
sys.path.append(str(Path.cwd().parent))

from src.data_ingestion import load_actions

In [4]:
df = load_actions("AAPL", "2018-01-01", "2024-12-31")

d:\Projetos\ProjetosPessoais\FIAP-Pos\tech-challenge\previsao_acoes\src\data_ingestion.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [5]:
print(df.head())

       close       high        low       open     volume   ano  mes  dia
0  40.341881  40.351250  39.639305  39.850080  102223600  2018    1    2
1  40.334862  40.878189  40.271633  40.405120  118071600  2018    1    3
2  40.522217  40.625262  40.299735  40.407462   89738400  2018    1    4
3  40.983589  41.070239  40.526916  40.618250   94640000  2018    1    5
4  40.831341  41.126421  40.732977  40.831341   82271200  2018    1    8


## Preparando os dados de treinamento para a criação de features e targets

In [19]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#### Inicializando o MinMaxScaler para escalar os dados entre 0 e 1

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

#### Escalando os dados para treino

In [33]:
scaled_data = scaler.fit_transform(df)

#### Definindo o tamanho dos dados de treinamento (80% dos dados)

In [34]:
training_data_len = int(np.ceil(len(scaled_data) * 80))

#### Separando os dados de treinamento

In [35]:
training_data = scaled_data[0:int(training_data_len), : ]

#### Criando listas vazias para as features(x_train) e o target(y_train)

In [36]:
x_train = []
y_train = []

#### Preenchendo o x_train com 60 dias de dados e y_train com os seguintes dias de preço de fechamento

In [37]:
for i in range(60, len(training_data)):
    x_train.append(training_data[i-60:i, 0])
    y_train.append(training_data[i, 0])

#### Convertendo as listas em arrays numpy para modelo de treinamento

In [38]:
x_train, y_train = np.array(x_train), np.array(y_train)

#### Remodelando o x_train para ser [samples, time_steps, features] requeridos pelo LSTM

In [39]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))